In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

In [2]:
scenarios = [
    #{"model": "image", "pathway": "SSP2-Base", "year": 2020},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2022},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2025},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2030},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2040},
    {"model": "image", "pathway": "SSP2-Base", "year": 2050},
]

sps = Package("../datapackage.json")

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[sps,],
        use_multiprocessing=False
)

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [3]:
ndb.update("external")

Updating: external:   0%|                       | 0/1 [00:00<?, ?it/s]

Regionalizing electricity production, oil, electricity, high voltage, RoW
not found in regions
Adding electricity production, oil, CH to the database
Regionalizing electricity production, compressed air energy storage, electricity, high voltage, RER
not found in regions
Adding electricity production, compressed air energy storage, CH to the database
Regionalizing heat and power co-generation, natural gas, conventional power plant, 100MW electrical, electricity, high voltage, RoW
not found in regions
Adding heat and power co-generation, natural gas, conventional power plant, 100MW electrical, CH to the database
Regionalizing carbon dioxide, captured at wood burning power plant 20 MW post, pipeline 200km, storage 1000m, carbon dioxide, captured at wood burning power plant 20 MW post, pipeline 200km, storage 1000m, RER
not found in regions
Adding carbon dioxide, captured at wood burning power plant 20 MW post, pipeline 200km, storage 1000m, CH to the database
Regionalizing electricity pro

Updating: external: 100%|███████████████| 1/1 [00:04<00:00,  4.83s/it]

Done!



In [4]:
for ds in ndb.scenarios[0]["database"]:
    if "SPS" in ds["name"]:
        for e in ds["exchanges"]:
            print(e["name"][:40], e["amount"], e.get("location"), e["unit"])
        print()

market for electricity, high voltage (SP 1 CH kilowatt hour
electricity production, hydro, run-of-ri 0.2098645143070873 CH kilowatt hour
electricity production, hydro, reservoir 0.24128864212686676 CH kilowatt hour
electricity production, hydro, pumped st 0.044427203948210384 CH kilowatt hour
electricity production, natural gas, 10M 5.227744089657459e-05 CH kilowatt hour
heat and power co-generation, natural ga 0.000321700668677393 CH kilowatt hour
heat and power co-generation, natural ga 2.404222733862657e-06 CH kilowatt hour
electricity production, at co-generation 1.3897630817602434e-06 CH kilowatt hour
heat and power co-generation, natural ga 0.0007319337772699444 CH kilowatt hour
heat and power co-generation, natural ga 5.906200050120565e-05 CH kilowatt hour
heat and power co-generation, biogas, ga 0.009267434940539356 CH kilowatt hour
electricity production, from incineratio 0.00775758732387819 CH kilowatt hour
electricity production, from incineratio 0.0011658322926844514 CH kil

In [5]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"] == "electricity supply, high voltage, from vanadium-redox flow battery system" and ds["unit"] == "kilowatt hour":
        print(list(ds.keys()))
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"))
        print()

['location', 'comment', 'production amount', 'source', 'reference product', 'type', 'unit', 'name', 'exchanges', 'worksheet name', 'database', 'code', 'regions', 'technosphere filters', 'biosphere filters', 'production volume variable']
electricity supply, high voltage, from vanadium-redox flow battery system 1 RER
market group for electricity, high voltage 1.33 RER
vanadium-redox flow battery system assembly, 8.3 megawatt hour 2.0408163265306123e-08 RER
vanadium-redox flow battery stack assembly 2.0408163265306123e-08 RER
treatment of stack, for VRFB system -0.00041 RER
treatment of electrolyte tank, for VRFB system -0.00066 RER
treatment of electrolyte solution, for VRFB system -0.00555 RER
treatment of periphericals, for VRFB system -0.0001 RER

['location', 'comment', 'production amount', 'source', 'reference product', 'type', 'unit', 'name', 'exchanges', 'worksheet name', 'database', 'code', 'regions', 'technosphere filters', 'biosphere filters', 'production volume variable', 'pro

In [4]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev.


In [6]:
ndb.write_db_to_brightway("test SPS1 7")

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Title: Writing activities to SQLite3 database:
  Started: 02/21/2024 12:48:43
  Finished: 02/21/2024 12:49:05
  Total time elapsed: 00:00:22
  CPU %: 95.40
  Memory %: 7.94
Created database: test SPS1 7
Generate scenario report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev.


In [14]:
ndb.scenarios[0]["external data"][0]["production volume"].coords["variables"].values

array(['Electricity generation|Electricity storage (non hydro)|Batteries in Services sector',
       'Electricity generation|Electricity storage (non hydro)|Batteries in industry',
       'Electricity generation|Electricity storage (non hydro)|Batteries in residential multi-family houses',
       'Electricity generation|Electricity storage (non hydro)|Batteries in residentiial single family houses',
       'Electricity generation|Electricity storage (non hydro)|Battery storage large size centralised',
       'Electricity generation|Electricity storage (non hydro)|Battery storage medium size centralised',
       'Electricity generation|Electricity storage (non hydro)|CAES',
       'Electricity generation|Geothermal|Deep geothermal',
       'Electricity generation|Hydrogen|CHP Fuel Cell Hydrogen (for District Heating)',
       'Electricity generation|Hydrogen|Industrial CHPs fuel cells (hydrogen)',
       'Electricity generation|Hydro|Hydro Large',
       'Electricity generation|Hydro|Hy

In [8]:
sps.errors

[]